In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, LabelEncoder
#from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

class LassoReg:
    def __init__(self, alpha=1.0, max_iter=100, tol=0.5):
        self.alpha = alpha
        self.max_iter = max_iter
        self.tol = tol
        self.coef_ = None
        self.intercept_ = None
    def fit(self, X, y):
      n_samples, n_features = X.shape
      self.coef_ = np.zeros(n_features)
      self.intercept_ = 0.0

      Transpose = np.dot(X.T, X)

      for _ in range(self.max_iter):
        prev_coef = np.copy(self.coef_)
        prev_intercept = self.intercept_

        residuals = y - np.dot(X, self.coef_)
        self.intercept_ = np.mean(residuals) if n_samples > 1 else residuals[0]

        for j in range(n_features):
            rho_j = np.dot(X[:, j], residuals + np.dot(X, self.coef_) - X[:, j] * self.coef_[j])
            if rho_j < -self.alpha / 2:
                self.coef_[j] = (rho_j + self.alpha / 2) / Transpose[j, j]
            elif rho_j > self.alpha / 2:
                self.coef_[j] = (rho_j - self.alpha / 2) / Transpose[j, j]
            else:
                self.coef_[j] = 0.0

        coef_change = np.max(np.abs(self.coef_ - prev_coef))
        intercept_change = np.abs(self.intercept_ - prev_intercept)
        if coef_change < self.tol and intercept_change < self.tol:
            break


    def predict(self, X):
        return np.dot(X, self.coef_) + self.intercept_

In [2]:
file_path = '/Users/ishikamanghwani/desktop/cardetails_csv.csv'
data = pd.read_csv(file_path)
data.head()
data.columns
data_encoded = pd.get_dummies(data, columns=['fuel', 'seller_type', 'transmission', 'owner'])
data_encoded.head()

,name,year,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti 800 AC,2007,70000,60000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
1,Maruti Wagon R LXI Minor,2007,50000,135000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
2,Hyundai Verna 1.6 SX,2012,100000,600000,False,True,False,False,False,False,True,False,False,True,True,False,False,False,False
3,Datsun RediGO T Option,2017,46000,250000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
4,Honda Amaze VX i-DTEC,2014,141000,450000,False,True,False,False,False,False,True,False,False,True,False,False,True,False,False


In [3]:
X = data_encoded.drop(columns=['selling_price','name'])
y = data_encoded['selling_price']

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

X_df = pd.DataFrame(X)
y_df = pd.Series(y)

nan_mask_X = X_df.isna().any(axis=1)
nan_mask_y = y_df.isna()

nan_mask_combined = nan_mask_X | nan_mask_y

X_clean = X_df[~nan_mask_combined]
y_clean = y_df[~nan_mask_combined]

isolation_forest = IsolationForest(contamination=0.1, random_state=42)  
outlier_mask = isolation_forest.fit_predict(X_clean) != -1  

X_clean = X_clean[outlier_mask]
y_clean = y_clean[outlier_mask]

y_clean = y_clean[:X_clean.shape[0]]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)
X_scaled_df = pd.DataFrame(X_scaled)

print("Original data shape:", X.shape)
print("Cleaned data shape:", X_clean.shape)
print("Scaled data shape:", X_scaled_df.shape)
print("Scaled feature shape", y_clean.shape)

Original data shape: (4340, 17)
Cleaned data shape: (3906, 17)
Scaled data shape: (3906, 17)
Scaled feature shape (3906,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y_clean, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Training data shape: (3124, 17)
Testing data shape: (782, 17)
Training target shape: (3124,)
Testing target shape: (782,)


In [6]:
data_encoded.head()

,name,year,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti 800 AC,2007,70000,60000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
1,Maruti Wagon R LXI Minor,2007,50000,135000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
2,Hyundai Verna 1.6 SX,2012,100000,600000,False,True,False,False,False,False,True,False,False,True,True,False,False,False,False
3,Datsun RediGO T Option,2017,46000,250000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
4,Honda Amaze VX i-DTEC,2014,141000,450000,False,True,False,False,False,False,True,False,False,True,False,False,True,False,False


In [7]:
print("X train y train \n")
X_train
y_train

X train y train 



271      130000
3409     160000
3051     400000
2632     200000
3634     350000
         ...   
1278     450000
1455     600000
975     1770000
3908     280000
3541     180000
Name: selling_price, Length: 3124, dtype: int64

In [8]:
print("Training data shape:", X_train.shape)
print("Training target shape:", y_train.shape)

Training data shape: (3124, 17)
Training target shape: (3124,)


In [9]:
#numpy
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

lasso_model = LassoReg(alpha=1.0)
lasso_model.fit(X_train_np, y_train_np)

predictions = lasso_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

Mean Squared Error: 1.4316395686226122e+33


In [36]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)
X_train_tf, X_temp, y_train_tf, y_temp = train_test_split(X_scaled_df, y_clean, test_size=0.2, random_state=42)
X_val, X_test_tf, y_val, y_test_tf = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [37]:
data_encoded.head()

,name,year,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Automatic,transmission_Manual,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti 800 AC,2007,70000,60000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
1,Maruti Wagon R LXI Minor,2007,50000,135000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
2,Hyundai Verna 1.6 SX,2012,100000,600000,False,True,False,False,False,False,True,False,False,True,True,False,False,False,False
3,Datsun RediGO T Option,2017,46000,250000,False,False,False,False,True,False,True,False,False,True,True,False,False,False,False
4,Honda Amaze VX i-DTEC,2014,141000,450000,False,True,False,False,False,False,True,False,False,True,False,False,True,False,False


In [38]:
X_train_tf,y_train_tf

(            0         1         2         3    4         5         6   \
 247  -1.757907  0.302270 -0.057787 -1.010811  0.0 -0.069915  1.027516   
 3055 -0.035529  0.302270 -0.057787 -1.010811  0.0 -0.069915  1.027516   
 2717  0.702634 -0.468771 -0.057787  0.989305  0.0 -0.069915 -0.973221   
 2334 -0.281583  0.551541 -0.057787  0.989305  0.0 -0.069915 -0.973221   
 3259  0.702634 -0.695254 -0.057787  0.989305  0.0 -0.069915 -0.973221   
 ...        ...       ...       ...       ...  ...       ...       ...   
 1130  0.456580  0.362011 -0.057787  0.989305  0.0 -0.069915 -0.973221   
 1294 -0.281583  0.302270 -0.057787  0.989305  0.0 -0.069915 -0.973221   
 860   0.948688 -0.916926 -0.057787  0.989305  0.0 -0.069915 -0.973221   
 3507  0.210526 -0.251910 -0.057787 -1.010811  0.0 -0.069915  1.027516   
 3174 -1.757907  3.459656 -0.057787  0.989305  0.0 -0.069915 -0.973221   
 
             7         8         9         10        11        12        13  \
 247  -0.507510  0.521439 -0.09

In [46]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=1.0, max_iter=10000)  
lasso_reg.fit(X_train_tf, y_train_tf)
lasso_pred = lasso_reg.predict(X_test_tf)

lasso_mse = mean_squared_error(y_test_tf, lasso_pred)
print(f'Lasso Regression MSE: {lasso_mse}')

Lasso Regression MSE: 176112156968.7864


In [44]:
import tensorflow as tf

# Define your model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_tf.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.legacy.Adam(), metrics=['mean_squared_error'])

history = model.fit(X_train_tf, y_train_tf, epochs=1000, batch_size=100, validation_data=(X_val, y_val))

test_loss, test_mse = model.evaluate(X_test_tf, y_test_tf)
print(f'Test MSE: {test_mse}')

Epoch 1/1000
32/32 [==============================] - 0s 2ms/step - loss: 422774931456.0000 - mean_squared_error: 422774931456.0000 - val_loss: 315947057152.0000 - val_mean_squared_error: 315947057152.0000
Epoch 2/1000
32/32 [==============================] - 0s 566us/step - loss: 422762414080.0000 - mean_squared_error: 422762414080.0000 - val_loss: 315929821184.0000 - val_mean_squared_error: 315929821184.0000
Epoch 3/1000
32/32 [==============================] - 0s 560us/step - loss: 422726598656.0000 - mean_squared_error: 422726598656.0000 - val_loss: 315884994560.0000 - val_mean_squared_error: 315884994560.0000
Epoch 4/1000
32/32 [==============================] - 0s 639us/step - loss: 422642745344.0000 - mean_squared_error: 422642745344.0000 - val_loss: 315788787712.0000 - val_mean_squared_error: 315788787712.0000
Epoch 5/1000
32/32 [==============================] - 0s 596us/step - loss: 422479953920.0000 - mean_squared_error: 422479953920.0000 - val_loss: 315615281152.0000 - val_

32/32 [==============================] - 0s 585us/step - loss: 126758920192.0000 - mean_squared_error: 126758920192.0000 - val_loss: 90812563456.0000 - val_mean_squared_error: 90812563456.0000
Epoch 80/1000
32/32 [==============================] - 0s 592us/step - loss: 126242775040.0000 - mean_squared_error: 126242775040.0000 - val_loss: 90437296128.0000 - val_mean_squared_error: 90437296128.0000
Epoch 81/1000
32/32 [==============================] - 0s 577us/step - loss: 125790945280.0000 - mean_squared_error: 125790945280.0000 - val_loss: 90035912704.0000 - val_mean_squared_error: 90035912704.0000
Epoch 82/1000
32/32 [==============================] - 0s 579us/step - loss: 125343195136.0000 - mean_squared_error: 125343195136.0000 - val_loss: 89685819392.0000 - val_mean_squared_error: 89685819392.0000
Epoch 83/1000
32/32 [==============================] - 0s 570us/step - loss: 124865257472.0000 - mean_squared_error: 124865257472.0000 - val_loss: 89302843392.0000 - val_mean_squared_err

32/32 [==============================] - 0s 619us/step - loss: 107205206016.0000 - mean_squared_error: 107205206016.0000 - val_loss: 72826994688.0000 - val_mean_squared_error: 72826994688.0000
Epoch 158/1000
32/32 [==============================] - 0s 608us/step - loss: 107105640448.0000 - mean_squared_error: 107105640448.0000 - val_loss: 72689000448.0000 - val_mean_squared_error: 72689000448.0000
Epoch 159/1000
32/32 [==============================] - 0s 582us/step - loss: 107023859712.0000 - mean_squared_error: 107023859712.0000 - val_loss: 72612995072.0000 - val_mean_squared_error: 72612995072.0000
Epoch 160/1000
32/32 [==============================] - 0s 597us/step - loss: 106943520768.0000 - mean_squared_error: 106943520768.0000 - val_loss: 72549400576.0000 - val_mean_squared_error: 72549400576.0000
Epoch 161/1000
32/32 [==============================] - 0s 573us/step - loss: 106874535936.0000 - mean_squared_error: 106874535936.0000 - val_loss: 72410251264.0000 - val_mean_squared

32/32 [==============================] - 0s 554us/step - loss: 102594273280.0000 - mean_squared_error: 102594273280.0000 - val_loss: 67074785280.0000 - val_mean_squared_error: 67074785280.0000
Epoch 236/1000
32/32 [==============================] - 0s 582us/step - loss: 102566330368.0000 - mean_squared_error: 102566330368.0000 - val_loss: 66988236800.0000 - val_mean_squared_error: 66988236800.0000
Epoch 237/1000
32/32 [==============================] - 0s 558us/step - loss: 102491291648.0000 - mean_squared_error: 102491291648.0000 - val_loss: 66956705792.0000 - val_mean_squared_error: 66956705792.0000
Epoch 238/1000
32/32 [==============================] - 0s 591us/step - loss: 102455664640.0000 - mean_squared_error: 102455664640.0000 - val_loss: 66914230272.0000 - val_mean_squared_error: 66914230272.0000
Epoch 239/1000
32/32 [==============================] - 0s 575us/step - loss: 102400606208.0000 - mean_squared_error: 102400606208.0000 - val_loss: 66870546432.0000 - val_mean_squared

32/32 [==============================] - 0s 592us/step - loss: 99548332032.0000 - mean_squared_error: 99548323840.0000 - val_loss: 63395098624.0000 - val_mean_squared_error: 63395098624.0000
Epoch 314/1000
32/32 [==============================] - 0s 596us/step - loss: 99502071808.0000 - mean_squared_error: 99502071808.0000 - val_loss: 63379435520.0000 - val_mean_squared_error: 63379435520.0000
Epoch 315/1000
32/32 [==============================] - 0s 607us/step - loss: 99459547136.0000 - mean_squared_error: 99459547136.0000 - val_loss: 63342419968.0000 - val_mean_squared_error: 63342419968.0000
Epoch 316/1000
32/32 [==============================] - 0s 603us/step - loss: 99429007360.0000 - mean_squared_error: 99429007360.0000 - val_loss: 63312814080.0000 - val_mean_squared_error: 63312814080.0000
Epoch 317/1000
32/32 [==============================] - 0s 587us/step - loss: 99399548928.0000 - mean_squared_error: 99399548928.0000 - val_loss: 63264210944.0000 - val_mean_squared_error: 63

...

Epoch 833/1000
32/32 [==============================] - 0s 613us/step - loss: 94245396480.0000 - mean_squared_error: 94245388288.0000 - val_loss: 57269248000.0000 - val_mean_squared_error: 57269248000.0000
Epoch 834/1000
32/32 [==============================] - 0s 598us/step - loss: 94243758080.0000 - mean_squared_error: 94243758080.0000 - val_loss: 57263493120.0000 - val_mean_squared_error: 57263493120.0000
Epoch 835/1000
32/32 [==============================] - 0s 583us/step - loss: 94251532288.0000 - mean_squared_error: 94251532288.0000 - val_loss: 57259343872.0000 - val_mean_squared_error: 57259343872.0000
Epoch 836/1000
32/32 [==============================] - 0s 576us/step - loss: 94204747776.0000 - mean_squared_error: 94204747776.0000 - val_loss: 57246674944.0000 - val_mean_squared_error: 57246674944.0000
Epoch 837/1000
32/32 [==============================] - 0s 581us/step - loss: 94229159936.0000 - mean_squared_error: 94229159936.0000 - val_loss: 57227902976.0000 - val_mean_sq

Epoch 873/1000
32/32 [==============================] - 0s 594us/step - loss: 94130667520.0000 - mean_squared_error: 94130667520.0000 - val_loss: 57103478784.0000 - val_mean_squared_error: 57103478784.0000
Epoch 874/1000
32/32 [==============================] - 0s 582us/step - loss: 94138335232.0000 - mean_squared_error: 94138335232.0000 - val_loss: 57106661376.0000 - val_mean_squared_error: 57106661376.0000
Epoch 875/1000
32/32 [==============================] - 0s 582us/step - loss: 94153179136.0000 - mean_squared_error: 94153179136.0000 - val_loss: 57098866688.0000 - val_mean_squared_error: 57098866688.0000
Epoch 876/1000
32/32 [==============================] - 0s 579us/step - loss: 94154915840.0000 - mean_squared_error: 94154915840.0000 - val_loss: 57104498688.0000 - val_mean_squared_error: 57104498688.0000
Epoch 877/1000
32/32 [==============================] - 0s 557us/step - loss: 94136442880.0000 - mean_squared_error: 94136434688.0000 - val_loss: 57108123648.0000 - val_mean_sq

Epoch 913/1000
32/32 [==============================] - 0s 597us/step - loss: 94086209536.0000 - mean_squared_error: 94086209536.0000 - val_loss: 57020403712.0000 - val_mean_squared_error: 57020403712.0000
Epoch 914/1000
32/32 [==============================] - 0s 595us/step - loss: 94057766912.0000 - mean_squared_error: 94057766912.0000 - val_loss: 57022484480.0000 - val_mean_squared_error: 57022484480.0000
Epoch 915/1000
32/32 [==============================] - 0s 585us/step - loss: 94090379264.0000 - mean_squared_error: 94090379264.0000 - val_loss: 57023692800.0000 - val_mean_squared_error: 57023692800.0000
Epoch 916/1000
32/32 [==============================] - 0s 577us/step - loss: 94064025600.0000 - mean_squared_error: 94064017408.0000 - val_loss: 57006923776.0000 - val_mean_squared_error: 57006923776.0000
Epoch 917/1000
32/32 [==============================] - 0s 604us/step - loss: 94071808000.0000 - mean_squared_error: 94071808000.0000 - val_loss: 57011892224.0000 - val_mean_sq

Epoch 953/1000
32/32 [==============================] - 0s 610us/step - loss: 94012473344.0000 - mean_squared_error: 94012473344.0000 - val_loss: 56909168640.0000 - val_mean_squared_error: 56909168640.0000
Epoch 954/1000
32/32 [==============================] - 0s 601us/step - loss: 93998186496.0000 - mean_squared_error: 93998186496.0000 - val_loss: 56897187840.0000 - val_mean_squared_error: 56897187840.0000
Epoch 955/1000
32/32 [==============================] - 0s 571us/step - loss: 94006190080.0000 - mean_squared_error: 94006190080.0000 - val_loss: 56894898176.0000 - val_mean_squared_error: 56894898176.0000
Epoch 956/1000
32/32 [==============================] - 0s 594us/step - loss: 94001020928.0000 - mean_squared_error: 94001020928.0000 - val_loss: 56904208384.0000 - val_mean_squared_error: 56904208384.0000
Epoch 957/1000
32/32 [==============================] - 0s 571us/step - loss: 93966917632.0000 - mean_squared_error: 93966917632.0000 - val_loss: 56905650176.0000 - val_mean_sq

Epoch 993/1000
32/32 [==============================] - 0s 580us/step - loss: 93981990912.0000 - mean_squared_error: 93981990912.0000 - val_loss: 56828227584.0000 - val_mean_squared_error: 56828227584.0000
Epoch 994/1000
32/32 [==============================] - 0s 611us/step - loss: 93951901696.0000 - mean_squared_error: 93951901696.0000 - val_loss: 56832831488.0000 - val_mean_squared_error: 56832831488.0000
Epoch 995/1000
32/32 [==============================] - 0s 593us/step - loss: 93933772800.0000 - mean_squared_error: 93933772800.0000 - val_loss: 56838221824.0000 - val_mean_squared_error: 56838221824.0000
Epoch 996/1000
32/32 [==============================] - 0s 591us/step - loss: 93940744192.0000 - mean_squared_error: 93940744192.0000 - val_loss: 56840716288.0000 - val_mean_squared_error: 56840716288.0000
Epoch 997/1000
32/32 [==============================] - 0s 584us/step - loss: 93936459776.0000 - mean_squared_error: 93936459776.0000 - val_loss: 56836870144.0000 - val_mean_sq